## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import re
from lxml import etree
import json
import torch
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances,manhattan_distances
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from sitemaps_utils import *

In [62]:
dic = {}


In [63]:
dic["0"] = {'upvotes': 0, 'downvotes': 0}
dic["1"] = {'upvotes': 1, 'downvotes': 0}
dic["2"] = {'upvotes': 2, 'downvotes': 0}

In [74]:
most_upvoted = sorted(dic.items(), key=lambda x: x[1]['upvotes'], reverse=True)
sorted_upvoted_idxs = [i[0] for idx, i in enumerate(most_upvoted) if i[1]["upvotes"] > 0]
related_articles = []
prefrences = ["sport","weather"]
for news_idx in sorted_upvoted_idxs:
    if articles_df.topic[int(news_idx)] not in prefrences:
            print("Select Preferences please")
            continue
    related_articles.append(I[int(news_idx)][1:])

#for articles in related_articles:
      
    
    
          


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Data Collection

#### BBC

In [38]:
BBC_news_sitemaps = ["https://www.bbc.com/sitemaps/https-sitemap-com-news-1.xml",
                     "https://www.bbc.com/sitemaps/https-sitemap-com-news-2.xml",
                     "https://www.bbc.com/sitemaps/https-sitemap-com-news-3.xml"]

sky_news_sitemaps = ["https://news.sky.com/sitemap/sitemap-news.xml",
                    "https://www.skysports.com/sitemap/sitemap-news.xml"]

namespaces = {
    'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9',
    'news': 'http://www.google.com/schemas/sitemap-news/0.9'
}

urls = {}
# BBC News uses 'sitemap:lastmod' for the date tag
urls["bbc"] = Extract_todays_urls_from_sitemaps(BBC_news_sitemaps, namespaces, 'sitemap:lastmod')
print(f"Number of BBC URLs: {len(urls['bbc'])}")

# Sky News uses 'news:publication_date' for the date tag
urls["sky"] = Extract_todays_urls_from_sitemaps(sky_news_sitemaps, namespaces, 'news:publication_date')
print(f"Number of Sky URLs: {len(urls['sky'])}")


Number of BBC URLs: 429
Number of Sky URLs: 18


## Data Processing

In [39]:
bbc_topics_to_drop = {"pidgin", "hausa", "swahili", "naidheachdan","videos","cymrufyw"}
df_BBC = process_news_data(urls, "bbc", bbc_topics_to_drop)

sky_topics_to_drop = {"arabic", "urdu"}
df_Sky = process_news_data(urls, "sky", sky_topics_to_drop)


INFO:langid.langid:initializing identifier


------ bbc ------
Topic
news         125
sport         34
weather       24
newsround      5
Total        188

------ sky ------
Topic
story       10
football     3
tennis       1
racing       1
cricket      1
Total        16



In [4]:
titles to drop: ["One-minute World News",""]

title contains "weekly" : df_BBC[df_BBC['Title'].str.contains('weekly round-up', case=False)]


SyntaxError: invalid syntax (918239044.py, line 1)

In [ ]:
df_BBC[df_BBC['Title']=="Watch Newsround"]

,Url,Last Modified,Title,Topic
106,https://www.bbc.com/newsround/66176104,2024-05-25T08:25:50Z,Watch Newsround,newsround


In [ ]:
df_BBC[~df_BBC['Title'].str.contains('weekly round-up', case=False)].drop_duplicates("Title").reset_index(inplace=True)

In [ ]:
for i in df_BBC[~df_BBC['Title'].str.contains('weekly round-up', case=False)].drop_duplicates("Title").Title:
    print(i)

One-minute World News
South America weather forecast
General election 2024 poll tracker: How do the parties compare?
RideLondon: Gas leak near finish line in Maldon
Michael Gove steps down in mass exodus of MPs before election
Defence Secretary Lloyd Austin to transfers powers due to medical procedure
Childbirth: Wrexham woman left with stoma and PTSD
Neurodivergent and queer author on purging Llanelli memories
Football gossip: McKenna, Ten Hag, Pochettino, Maresca, Sancho, Salah, Kompany
Extinct ‘mountain jewel’ plant returned to wild - in secret location
Smartphone ban for kids is worth considering - MPs
Papua New Guinea landslide: Race to rescue villagers trapped
Royal Mail investigated by Ofcom for missing delivery targets
Bournemouth stabbing at beach leaves woman dead
South Africa elections: Some are losing faith in the ANC
Rudimental's Locksmith meets 10-year-old carer at festival
Gaza war: Israelis attack aid convoys sent for Palestinians
Britain's Got Talent: Comedian ‘embrace

In [ ]:
for i in df_BBC[df_BBC['Title'].str.contains('weekly round-up', case=False)].Title:
    print(i)

Sussex weekly round-up: 18 May - 24 May 2024 
Surrey weekly round-up: 18 May - 24 May 2024 
Kent weekly round-up: 18 May - 24 May 2024


## Data cleaning

In [ ]:
# news_sport = ["news","sport"]
# news_sport_urls = df_BBC[df_BBC.Topic.isin(news_sport)]
# news_sport_urls.reset_index(inplace=True)

In [40]:
def remove_elements(input_string: str) -> str:
    """
    This function removes all HTML tags and their content from the input string,
    and removes specific patterns such as "Published X hours ago" and "Image source".
    """
    # Parse the input string as HTML
    soup = BeautifulSoup(input_string, 'html.parser')
    
    # Remove <script>, <style>, and <picture> tags and their content
    for script in soup(["script", "style", "picture"]):
        script.decompose()
    
    # Extract text from the parsed HTML
    cleaned_string = soup.get_text(separator=' ', strip=True)

    # Define regular expressions to remove unwanted patterns
    patterns = [
        r'\bPublished.*?\bago\b',  # Matches "Published X hours ago"
        r'\bImage source\b',   # Matches "Image source, ..."
        r'\bImage caption\b',  # Matches "Image caption, ..."
        r'\bMedia caption\b',  # Matches "Media caption, ..."
        r'\bGetty Images\b', 
        r'\bBBC Wales News\b',         # Matches "BBC Wales News"
        r'\bPublished\s\d{1,2}\s\w+\b',
        #r'\bRelated Topics\b.*',
        r'\bRelated Internet\b.*', 
        r'\bBBC News Staff\b',
        r'\bFollow\s.*?\snews.*\b',
        r'\b\w+/\w+\b',
        r'Follow\sBBC.*'           
 
    ]

    # Remove the matched patterns from the text
    for pattern in patterns:
        cleaned_string = re.sub(pattern, '', cleaned_string, flags=re.DOTALL)
    
    # Additional cleanup: remove extra spaces and newlines
    cleaned_string = re.sub(r'["\'.,]+', '', cleaned_string)
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()
    cleaned_string = cleaned_string.replace("This video can not be played To play this video you need to enable JavaScript in your browser.","")
    string2 = "Sign up for our morning newsletter and get BBC News in your inbox."
    cleaned_string = cleaned_string.replace(string2,"")

    
    article = []
    for line in (i for i in cleaned_string.split("\n") if len(i) >= 10):
        article.append(line)

    return " ".join(article)

In [41]:
async def request_sentences_from_urls_async(urls, timeout=20):
    articles_df = pd.DataFrame(columns=["url","topic","title","content"])

    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls.Url, start=1):
            if (idx - 1) % 100 == 0:
                logging.info(f"\nProcessing URL {((idx - 1)//100)+1}/{(len(urls)//100)+1}")

            tasks.append(fetch_url(session, url, timeout))

        results = await asyncio.gather(*tasks)

        for idx, (url, result) in enumerate(zip(urls.Url, results), start=1):
            if result is None:
                continue

            try:
                tree = etree.HTML(result)
                article_element = tree.find(".//article")
                if article_element is not None:
                    outer_html = etree.tostring(article_element, encoding='unicode')
                    article_body = remove_elements(outer_html)
                    article = [line for line in article_body.split("\n") if len(line) >= 40]
                    articles_df.loc[idx - 1] = (urls["Url"][idx - 1],urls["Topic"][idx - 1],urls["Title"][idx - 1]," ".join(article))
                else:
                    # If no <article> element is found, try using BeautifulSoup with the specific ID
                    soup = BeautifulSoup(result, 'html.parser')
                    article_id = 'main-content'  # Replace with the actual ID you are targeting
                    article_element = soup.find(id=article_id)
                    if article_element:
                        article_body = remove_elements(str(article_element))
                        article = [line for line in article_body.split("\n") if len(line) >= 40]
                        articles_df.loc[idx - 1] = (urls["Url"][idx - 1],urls["Topic"][idx - 1],urls["Title"][idx - 1]," ".join(article))
                    else:
                        logging.warning(f"No article content found on the page with ID {article_id}.")
            except Exception as e:
                logging.error(f"Error extracting article content from {url}: error: {e}")


    return articles_df

In [42]:
timeout = 20  # Timeout value
async def main():
    articles = await request_sentences_from_urls_async(df_BBC, timeout)
    return articles
articles_df = await main()


INFO:root:
Processing URL 1/2
INFO:root:
Processing URL 2/2


## Embeding and similarity check

In [43]:
import torch
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import re

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

article_main_body = list(articles_df.content)

# Load tokenizer and model
tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)
if torch.cuda.device_count() > 1:
    encoder = torch.nn.DataParallel(encoder)

print("Using device:", device)

# Initialize an empty list to store embeddings
embeddings = []

# Process and encode each article body
for i, data in enumerate(article_main_body):
    # Tokenize with padding
    inputs = tokenizer("".join(data), return_tensors="pt", padding='max_length', truncation=True, max_length=512).to(device)  # Move inputs to GPU
    with torch.no_grad():  # No need to track gradients during inference
        embedding = encoder(**inputs).pooler_output
    embeddings.append(embedding)
    if i % 10 == 0:
        print(f"{i // 10}/{len(article_main_body) // 10}")

print("Encoding completed.")

c:\Users\amoha\anaconda3\envs\cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\amoha\anaconda3\envs\cuda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base and are newly initialized: ['bert_model.embeddings.LayerNorm.bias', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.word_embeddings.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.b

Using device: cuda
0/18
1/18
2/18
3/18
4/18
5/18
6/18
7/18
8/18
9/18
10/18
11/18
12/18
13/18
14/18
15/18
16/18
17/18
18/18
Encoding completed.


In [44]:
# Convert embeddings tensor to numpy arrays
embeddings_np = [embedding.cpu().numpy() for embedding in embeddings]

# Convert embeddings to float32
embeddings_np = [embedding.astype('float32') for embedding in embeddings_np]
embeddings_np = np.array(embeddings_np).reshape(len(embeddings_np), 768)
embeddings_list = [embedding for embedding in embeddings_np]

In [45]:
import pickle

articles_df["embedding"] = embeddings_list
articles_df = articles_df.drop_duplicates(subset='content', keep='first').reset_index(drop=True)

#content_embedding = (list(bbc_news.values()),embeddings_np)

with open('content_embedding.pkl', 'wb') as file:
    pickle.dump(articles_df, file)

print("Embeddings saved successfully.")

Embeddings saved successfully.


In [46]:
import faiss
import numpy as np



# Initialize the index
index = faiss.IndexFlatL2(embeddings_np.shape[1])  # Assuming embeddings are of the same dimension
embedding = [np.array(i) for i in articles_df.embedding]

# Add embeddings to the index
index.add(np.array(embedding))

# Search for similar embeddings
D, I = index.search(np.array(embedding), k=2)  # k=2 returns the closest two vectors (including self)

high_sim = []
titles = list(articles_df.title)

# To keep track of which titles have been printed
printed_titles = set()

for i in range(len(I)):
    title1 = titles[i]
    closest_index = I[i][1]  # I[i][0] will be the index of itself, so we take the second closest
    title2 = titles[closest_index]
    similarity_score = 1 - D[i][1]  # cosine similarity is 1 - L2 distance
    if title1 not in printed_titles and title2 not in printed_titles and similarity_score >-0.4:
        print(f"Title 1: '{title1}'\nTitle 2: '{title2}'\nSimilarity Score: {similarity_score}")
        high_sim.append((title1, title2, similarity_score))
        printed_titles.add(title1)
        printed_titles.add(title2)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


Title 1: 'Tornadoes and storms leave 15 dead across central US'
Title 2: 'Papua New Guinea: Thousands feared missing after landslide'
Similarity Score: -0.297607421875
Title 1: 'Berkshire's Big Picture: 27 May - 2 June 2024'
Title 2: 'Dorset's Big Picture: 27 May - 2 June 2024'
Similarity Score: 0.46630859375


Try other types of  clustering methods such as :

Density-based spatial clustering (DBSC) : a density-based clustering algorithm that works on the assumption that clusters are dense regions in space separated by regions of lower density. It groups 'densely grouped' data points into a single cluster.

SPECTURAL CLUSTERING : uses information from the eigenvalues (spectrum) of special matrices (i.e. Affinity Matrix, Degree Matrix and Laplacian Matrix) derived from the graph or the data set.
DBA - ALOCATION


# Next steps

### 1

1) **SKY :** some urls are the same article from sky news and sky sports. check if they have same text and remove if they do?

2) **SKY & BBC :** Make sure BBC and SKY only have news and sports sections that are important to us. 

3) choose more websites